# Preparación: El Dataset "Pesadilla"

In [11]:
import pandas as pd
import numpy as np

data_sucio = {
    'cod_transaccion': [' TR-2023_001 ', 'tr_2022-005', 'TR.2024.010 ', ' Tr-2023_002'],
    'cliente_contacto': ['Juan Perez | +57 300-123', 'Marta Diaz | 310.456', 'PEDRO RUIZ | 320 987', 'ana gomez|555-000'],
    'nota_venta': ['URGENTE: Enviar mañana', 'Stock bajo', 'urgente - revisar', 'Cliente VIP - enviar ya']
}

df_sucio = pd.DataFrame(data_sucio)
print("---  DATOS REALES (SUCIOS) ---")
print(df_sucio)

---  DATOS REALES (SUCIOS) ---
  cod_transaccion          cliente_contacto               nota_venta
0    TR-2023_001   Juan Perez | +57 300-123   URGENTE: Enviar mañana
1     tr_2022-005      Marta Diaz | 310.456               Stock bajo
2    TR.2024.010       PEDRO RUIZ | 320 987        urgente - revisar
3     Tr-2023_002         ana gomez|555-000  Cliente VIP - enviar ya


# Ejemplo 1: Normalización Extrema de Códigos

Teoría: Cuando los códigos vienen de diferentes sistemas, los separadores varían (_, -, .). Para que Pandas los reconozca como iguales, debemos "planchar" el texto: quitar espacios, poner todo igual y usar un solo separador.

In [12]:
#Creamos la copia para no perder los datos originales por si necesitamos verificar datos viejos
df_limpio = df_sucio.copy()

# 1. Limpiamos espacios y estandarizamos a mayúsculas
df_limpio['cod_transaccion'] = df_limpio['cod_transaccion'].str.strip().str.upper()

# 2. Reemplazamos cualquier separador raro(_,.) por un guión (-)
df_limpio['cod_transaccion'] = df_limpio['cod_transaccion'].str.replace('_','-').str.replace('.', '-')

print(df_limpio['cod_transaccion'])

0    TR-2023-001
1    TR-2022-005
2    TR-2024-010
3    TR-2023-002
Name: cod_transaccion, dtype: str


Explicación línea por línea:

    .str.strip().str.upper(): Encadenamos dos acciones. Primero borramos los espacios invisibles en los extremos y luego convertimos todo a MAYÚSCULAS para que "tr" y "TR" sean lo mismo.

    .str.replace('_', '-').str.replace('.', '-'): Buscamos los caracteres inconsistentes uno por uno y los convertimos en el estándar de la empresa (el guión). Así, "TR-2023_001" y "TR.2023.001" pasan a ser idénticos.

# Ejemplo 2: El Divisor Maestro (split con limpieza)

Teoría: A veces dos datos distintos (Nombre y Teléfono) vienen pegados en la misma celda con un separador como el "pipe" (|). Necesitamos separarlos y asegurarnos de que no queden espacios "basura" alrededor del corte.

In [13]:
# 1. Dividimos por el simbolo '|' y expandimos a columnas nuevas
# Usamos expand = True para que nos devuelva un DataFrame
info_separada = df_limpio['cliente_contacto'].str.split('|', expand=True)

# 2. Asignamos y limpiamos espacios que quedaron cerca del separador
df_limpio['nombre'] = info_separada[0].str.strip().str.title()
df_limpio['telefono'] = info_separada[1].str.strip()

print(df_limpio[['nombre', 'telefono']])

       nombre     telefono
0  Juan Perez  +57 300-123
1  Marta Diaz      310.456
2  Pedro Ruiz      320 987
3   Ana Gomez      555-000


Explicación línea por línea:

    .str.split('|', expand=True): Corta el texto cada vez que encuentra la barra vertical. El expand=True es la clave para que el resultado se convierta automáticamente en dos columnas separadas.

    info_separada[0].str.strip().str.title(): Como al separar "Juan | 300" queda un espacio después de Juan ("Juan "), usamos strip() para borrarlo y title() para que el nombre se vea profesional.

# Ejemplo 3: Extracción de Inteligencia (Slicing)

Teoría: En muchos códigos, la posición de los caracteres significa algo. En TR-2023-001, los caracteres del 3 al 7 son el año. Extraer esto es vital para reportes anuales.

In [14]:
# Extraemos el año basándonos en la posición de las letras
# El año empieza en el índice 3 y terminamos antes del 7
df_limpio['año_pago'] = df_limpio['cod_transaccion'].str[3:7]

print(df_limpio[['cod_transaccion', 'año_pago']])

  cod_transaccion año_pago
0     TR-2023-001     2023
1     TR-2022-005     2022
2     TR-2024-010     2024
3     TR-2023-002     2023


Explicación línea por línea:

    .str[3:7]: Es como usar una tijera. Le decimos a Pandas: "Salta las 3 primeras letras (T, R, -) y recorta lo que sigue hasta llegar a la posición 7". Esto extrae el "2023", "2022", etc., de forma masiva.

# Ejemplo 4: Clasificación por Palabras Clave (contains)

Teoría: En el mundo laboral, necesitas categorizar filas rápidamente. Si una nota dice "URGENTE", esa transacción debe ir a una lista prioritaria.

In [16]:
# Buscamos la palabra 'urgente' sin importar si está en mayúsculas o minúsculas
df_limpio['es_prioritario'] = df_limpio['nota_venta'].str.contains('urgente|ya|vip', case=False)

# Verificamos quiénes son prioridad
print(df_limpio[['nota_venta', 'es_prioritario']])

                nota_venta  es_prioritario
0   URGENTE: Enviar mañana            True
1               Stock bajo           False
2        urgente - revisar            True
3  Cliente VIP - enviar ya            True


Explicación línea por línea:

    .str.contains('urgente', case=False): Escanea toda la columna nota_venta. El parámetro case=False es fundamental porque permite encontrar tanto "URGENTE" como "urgente". Devuelve True si la palabra existe en cualquier parte de la frase, funcionando como un filtro inteligente.